# Notebook for feature engineering II

In [1]:
%load_ext autoreload
%autoreload 2

#Imports
import pandas as pd
import requests
import os
from tqdm.notebook import tqdm
import pickle
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from  ift6758.data.data_acquisition import Season
from ift6758.visualizations.simple_visualization import Utilities 

In [2]:
season2015 = Season(2015,"../ift6758/data")
season2016 = Season(2016,"../ift6758/data")
season2017 = Season(2017,"../ift6758/data")
season2018 = Season(2018,"../ift6758/data")
season2019 = Season(2019,"../ift6758/data")

In [3]:
df_2015 = season2015.clean_data()
df_2016 = season2016.clean_data()
df_2017 = season2017.clean_data()
df_2018 = season2018.clean_data()
df_2019 = season2019.clean_data()

File already Exists, loading from ../ift6758/data/PICKLE//2015_clean.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2016.pkl
Len of games_list in 2016 is 1332


/home/yassir/Maitrise UdeM/IFT-6758/Projet/ift6758-project-template-main/ift6758/data/data_acquisition.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil_event['shooter'],df_fil_event['goalie'] = zip(*df_fil_event["players"].map(important_players)) ## Choosing Goalie and shooter
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1299/1299 [08:19<00:00,  2.60it/s]


Len of games_list in 2017 is 1376


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1299/1299 [10:56<00:00,  1.98it/s]


Len of games_list in 2018 is 1375
File already Exists, loading from ../ift6758/data/PICKLE//2019_clean.pkl


In [4]:
df_2019.head(10)

,result.event,gamePk,team.name,about.period,about.periodTime,about.periodType,about.periodTimeRemaining,coordinates.x,coordinates.y,result.secondaryType,result.emptyNet,result.strength.name,shooter,goalie
0,Shot,2019010001,Vegas Golden Knights,1,00:52,REGULAR,19:08,-34.0,19.0,Wrist Shot,NaN,NaN,Reilly Smith,Adin Hill
1,Shot,2019010001,Arizona Coyotes,1,01:12,REGULAR,18:48,67.0,-15.0,Snap Shot,NaN,NaN,Robbie Russo,Malcolm Subban
2,Shot,2019010001,Vegas Golden Knights,1,01:46,REGULAR,18:14,-36.0,32.0,Snap Shot,NaN,NaN,Zach Whitecloud,Adin Hill
3,Goal,2019010001,Arizona Coyotes,1,04:00,REGULAR,16:00,42.0,-19.0,Snap Shot,False,Even,Ilya Lyubushkin,Malcolm Subban
4,Goal,2019010001,Vegas Golden Knights,1,04:39,REGULAR,15:21,-71.0,-3.0,Wrist Shot,False,Even,Max Pacioretty,Adin Hill
5,Shot,2019010001,Arizona Coyotes,1,05:47,REGULAR,14:13,54.0,-31.0,Snap Shot,NaN,NaN,Nate Schnarr,Malcolm Subban
6,Shot,2019010001,Arizona Coyotes,1,05:56,REGULAR,14:04,49.0,24.0,Snap Shot,NaN,NaN,Beau Bennett,Malcolm Subban
7,Goal,2019010001,Arizona Coyotes,1,09:08,REGULAR,10:52,52.0,-31.0,Snap Shot,False,Even,Vinnie Hinostroza,Malcolm Subban
8,Shot,2019010001,Vegas Golden Knights,1,11:05,REGULAR,08:55,-43.0,-32.0,Wrist Shot,NaN,NaN,Curtis McKenzie,Adin Hill
9,Shot,2019010001,Arizona Coyotes,1,11:29,REGULAR,08:31,53.0,-27.0,Backhand,NaN,NaN,Hudson Fasching,Malcolm Subban
